# Strategy Validation

The strategy to be validated consists of several steps:

1. Structured query selection: In the first step, papers are selected based on a structured search query. The goal is to identify papers that reference datasets within a specific research domain.

2. Title + Abstract screening with GEMINI: Titles and abstracts are then screened using GEMINI to determine which papers meet predefined inclusion and exclusion criteria. At this stage, an initial extraction of datasets is also performed if they are explicitly mentioned.

3. Full-text screening with GEMINI: Finally, for papers that have not been excluded and for which datasets have not yet been associated, an additional screening is applied to the full text to extract any remaining relevant dataset information.

## Step 1: Structured Query Selection

The following queries are used to select papers for the validation study:

### IEEE Xplore

("All Metadata": "computer vision" OR "deep learning")

AND ("All Metadata": "action quality" OR AQA)

AND ("All Metadata": dataset OR "data set" OR database OR "data catalogue" OR "data repository" OR "data sharing" OR "open data")

To ADD Filters:

Year: 2013-2024

Document Type: Article + Conference Paper

-> Language Filter doesn't exists


### Scopus

TITLE-ABS-KEY ("computer vision" OR "deep learning")

AND TITLE-ABS-KEY ("action quality" OR AQA)

AND TITLE-ABS-KEY (dataset OR "data set" OR database OR "data catalogue" OR "data repository" OR "data sharing" OR "open data")

AND PUBYEAR > 2012 AND PUBYEAR < 2025

AND (LIMIT-TO(DOCTYPE,"ar") OR LIMIT-TO(DOCTYPE,"cp"))

AND (LIMIT-TO(LANGUAGE,"English"))


### Web of Science

TS=("computer vision" OR "deep learning")

AND TS=("action quality" OR AQA)

AND TS=(dataset OR "data set" OR database OR "data catalogue" OR "data repository" OR "data sharing" OR "open data")

AND PY=(2013-2024)

AND DT=(Article OR Proceedings Paper)

AND LA=(English)

In [2]:
import os
from collections import deque
from dotenv import load_dotenv
from google import genai
import pandas as pd
from pydantic import BaseModel, Field
import time
from typing import Annotated, Literal

folder_path = 'data/validation/'

ieee_file = folder_path + 'ieee_validation.csv'
scopus_file = folder_path + 'scopus_validation.csv'
wos_file = folder_path + 'wos_validation.xls'

## IEEE Xplore

In [3]:
df_ieee = pd.read_csv(ieee_file)
df_ieee

,Document Title,Authors,Author Affiliations,Publication Title,Date Added To Xplore,Publication Year,Volume,Issue,Start Page,End Page,...,Mesh_Terms,Article Citation Count,Patent Citation Count,Reference Count,License,Online Date,Issue Date,Meeting Date,Publisher,Document Identifier
0,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,"Nippon Telegraph and Telephone Corporation, Yo...",IEEE Access,10 Jul 2024,2024,12.0,NaN,92062,92072,...,NaN,1.0,NaN,55,CCBYNCND,4 Jul 2024,NaN,NaN,IEEE,IEEE Journals
1,Action Quality Assessment with Multi-scale Tem...,W. Wang; H. Wang; Y. Hao; Q. Wang,Faculty of Electronic Information and Electric...,2024 7th International Conference on Advanced ...,18 Jun 2024,2024,NaN,NaN,247,251,...,NaN,NaN,NaN,17,IEEE,18 Jun 2024,NaN,NaN,IEEE,IEEE Conferences
2,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,"NTT Media Intelligence Laboratories, NTT Corpo...",2021 IEEE International Conference on Image Pr...,23 Aug 2021,2021,NaN,NaN,1189,1193,...,NaN,6.0,NaN,27,IEEE,23 Aug 2021,NaN,NaN,IEEE,IEEE Conferences
3,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,"Department of Automation, Tsinghua University,...",2022 IEEE/CVF Conference on Computer Vision an...,27 Sep 2022,2022,NaN,NaN,2939,2948,...,NaN,86.0,NaN,50,IEEE,27 Sep 2022,NaN,NaN,IEEE,IEEE Conferences
4,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,"Shenzhen International Graduate School, Tsingh...",2023 IEEE/CVF Conference on Computer Vision an...,22 Aug 2023,2023,NaN,NaN,2405,2414,...,NaN,32.0,NaN,58,IEEE,22 Aug 2023,NaN,NaN,IEEE,IEEE Conferences
5,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,"School of Computing, University of Portsmouth,...",2023 International Conference on Machine Learn...,28 Nov 2023,2023,NaN,NaN,483,488,...,NaN,2.0,NaN,14,IEEE,28 Nov 2023,NaN,NaN,IEEE,IEEE Conferences
6,PE3DNet: A “Pull-Up” Action Quality Assessment...,R. Yang; X. Xiu; J. Wang; R. Wang,School of Electronic and Information Engineeri...,2024 7th International Conference on Pattern R...,14 Jan 2025,2024,NaN,NaN,742,747,...,NaN,NaN,NaN,28,IEEE,14 Jan 2025,NaN,NaN,IEEE,IEEE Conferences
7,PECoP: Parameter Efficient Continual Pretraini...,A. Dadashzadeh; S. Duan; A. Whone; M. Mirmehdi,"School of Computer Science, University of Bris...",2024 IEEE/CVF Winter Conference on Application...,9 Apr 2024,2024,NaN,NaN,42,52,...,NaN,17.0,NaN,51,IEEE,9 Apr 2024,NaN,NaN,IEEE,IEEE Conferences
8,Tai Chi Action Quality Assessment and Visual A...,J. Li; H. Hu; Q. Xing; X. Wang; J. Li; Y. Shen,"School of Sports Engineering, Beijing Sports U...",2022 IEEE 24th International Workshop on Multi...,22 Nov 2022,2022,NaN,NaN,1,6,...,NaN,6.0,NaN,26,IEEE,22 Nov 2022,NaN,NaN,IEEE,IEEE Conferences
9,A Survey of Video-based Action Quality Assessment,S. Wang; D. Yang; P. Zhai; Q. Yu; T. Suo; Z. S...,"Institute of AI & Robotics, Fudan University, ...",2021 International Conference on Networking Sy...,19 Apr 2022,2021,NaN,NaN,1,9,...,NaN,17.0,NaN,61,IEEE,19 Apr 2022,NaN,NaN,IEEE,IEEE Conferences


In [4]:
df_ieee = df_ieee[['DOI', 'Document Title', 'Authors', 'Publication Year', 'Abstract', 'Article Citation Count']]

df_ieee = df_ieee.rename(columns={
    'Document Title': 'Title',
    'Publication Year': 'Year',
    'Article Citation Count': 'Citations'
})

df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...,1.0
1,10.1109/ICAACE61206.2024.10548995,Action Quality Assessment with Multi-scale Tem...,W. Wang; H. Wang; Y. Hao; Q. Wang,2024,Action quality assessment is a more challengin...,NaN
2,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...,6.0
3,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...,86.0
4,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...,32.0
5,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...,2.0
6,10.1109/PRAI62207.2024.10827404,PE3DNet: A “Pull-Up” Action Quality Assessment...,R. Yang; X. Xiu; J. Wang; R. Wang,2024,“Pull-up” is a regular physical fitness test i...,NaN
7,10.1109/WACV57701.2024.00012,PECoP: Parameter Efficient Continual Pretraini...,A. Dadashzadeh; S. Duan; A. Whone; M. Mirmehdi,2024,The limited availability of labelled data in A...,17.0
8,10.1109/MMSP55362.2022.9949464,Tai Chi Action Quality Assessment and Visual A...,J. Li; H. Hu; Q. Xing; X. Wang; J. Li; Y. Shen,2022,Visual-based human action analysis is an impor...,6.0
9,10.1109/INSAI54028.2021.00029,A Survey of Video-based Action Quality Assessment,S. Wang; D. Yang; P. Zhai; Q. Yu; T. Suo; Z. S...,2021,Human action recognition and analysis have gre...,17.0


In [5]:
df_ieee = df_ieee.dropna(subset=['DOI'])
df_ieee = df_ieee.dropna(subset=['Citations'])
df_ieee

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...,1.0
2,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...,6.0
3,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...,86.0
4,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...,32.0
5,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...,2.0
7,10.1109/WACV57701.2024.00012,PECoP: Parameter Efficient Continual Pretraini...,A. Dadashzadeh; S. Duan; A. Whone; M. Mirmehdi,2024,The limited availability of labelled data in A...,17.0
8,10.1109/MMSP55362.2022.9949464,Tai Chi Action Quality Assessment and Visual A...,J. Li; H. Hu; Q. Xing; X. Wang; J. Li; Y. Shen,2022,Visual-based human action analysis is an impor...,6.0
9,10.1109/INSAI54028.2021.00029,A Survey of Video-based Action Quality Assessment,S. Wang; D. Yang; P. Zhai; Q. Yu; T. Suo; Z. S...,2021,Human action recognition and analysis have gre...,17.0
10,10.1109/CVPR52733.2024.01386,FineParser: A Fine-Grained Spatio-Temporal Act...,J. Xu; S. Yin; G. Zhao; Z. Wang; Y. Peng,2024,Existing action quality assessment (AQA) metho...,17.0
11,10.1109/CVPRW63382.2024.00324,FineRehab: A Multi-modality and Multi-task Dat...,J. Li; J. Xue; R. Cao; X. Du; S. Mo; K. Ran; Z...,2024,The assessment of rehabilitation exercises for...,8.0


### Total of 24 papers identified against 50 of the original paper
### *without DOMAIN filter, total of 53 papers

## Scopus

In [6]:
df_scopus = pd.read_csv(scopus_file)
df_scopus

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISSN,ISBN,CODEN,PubMed ID,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"A., Dadashzadeh, Amirhossein; S., Duan, Shucha...","Dadashzadeh, Amirhossein (57213067843); Duan, ...",57213067843; 58722692300; 6602648827; 7004105162,PECoP: Parameter Efficient Continual Pretraini...,2024,NaN,NaN,NaN,NaN,42.0,...,NaN,9798350318920,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85191968585
1,"R., Yang, Rui; X., Xiu, Xiaona; J., Wang, Jian...","Yang, Rui (58591741000); Xiu, Xiaona (59011186...",58591741000; 59011186000; 59863130600; 5954808...,PE3DNet: A 'Pull-Up' Action Quality Assessment...,2024,NaN,NaN,NaN,NaN,742.0,...,NaN,9798350350890,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85217223529
2,"W., Wang, Wei; H., Wang, Hongyu; Y., Hao, Ying...","Wang, Wei (57192615575); Wang, Hongyu (2203706...",57192615575; 22037060600; 57199703948; 5921058...,Action Quality Assessment with Multi-scale Tem...,2024,NaN,NaN,NaN,NaN,247.0,...,NaN,9798350361445,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85197889824
3,"T., Nagai, Takasuke; S., Takeda, Shoichiro; S....","Nagai, Takasuke (57203634284); Takeda, Shoichi...",57203634284; 57207765972; 57033228400; 5719042...,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,2024,IEEE Access,12,NaN,NaN,92062.0,...,21693536,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85197516873
4,"S., Sardari, Sara; B., Nakisa, Bahareh; S., Sh...","Sardari, Sara (57509554600); Nakisa, Bahareh (...",57509554600; 56156792800; 53985366000; 3551934...,Online Deep Squat Evaluation: Leveraging Subje...,2024,NaN,NaN,NaN,NaN,NaN,...,NaN,9798331519339,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-105010815473
5,"P., Lian, Puxiang; Z., Shao, Zhigang","Lian, Puxiang (58696312500); Shao, Zhigang (35...",58696312500; 35574802700,Improving action quality assessment with acros...,2023,Applied Intelligence,53,24,NaN,30443.0,...,15737497; 0924669X,9780511611445; 9780521884280,APITE,NaN,English,Article,Final,NaN,Scopus,2-s2.0-85176769140
6,"K., Zhou, Kanglei; Y., Ma, Yue; H.P., Shum, Hu...","Zhou, Kanglei (57205674291); Ma, Yue (57218566...",57205674291; 57218566782; 25032239300; 7401735847,Hierarchical Graph Convolutional Networks for ...,2023,IEEE Transactions on Circuits and Systems for ...,33,12,3281413,7749.0,...,10518215,NaN,ITCTE,NaN,English,Article,Final,All Open Access; Green Accepted Open Access; G...,Scopus,2-s2.0-85161002529
7,"N., Hao, Ning; S., Ruan, Sihan; Y., Song, Yihe...","Hao, Ning (57207816180); Ruan, Sihan (57226663...",57207816180; 57226663703; 57221792710; 5731641...,The Establishment of a precise intelligent eva...,2023,Heliyon,9,11,e21361,NaN,...,24058440,NaN,NaN,NaN,English,Article,Final,All Open Access; Gold Open Access; Green Final...,Scopus,2-s2.0-85174462280
8,"C.K., Ingwersen, Christian Keilstrup; A., Xarl...","Ingwersen, Christian Keilstrup (57205532216); ...",57205532216; 57955669900; 55329424200; 5652437...,Video-based Skill Assessment for Golf: Estimat...,2023,NaN,NaN,NaN,NaN,31.0,...,NaN,9798400702693,NaN,NaN,English,Conference paper,Final,All Open Access; Green Final Open Access; Gree...,Scopus,2-s2.0-85178297880
9,"Y., Liu, Yanchao; X., Cheng, Xina; T., Ikenaga...","Liu, Yanchao (58262316100); Cheng, Xina (56621...",58262316100; 56621799800; 8882572600,A Figure Skating Jumping Dataset for Replay-Gu...,2023,NaN,NaN,NaN,NaN,2437.0,...,NaN,9798400701085,NaN,NaN,English,Conference paper,Final,NaN,Scopus,2-s2.0-85179548737


In [7]:
df_scopus = df_scopus[['DOI', 'Title', 'Authors', 'Year', 'Abstract', 'Cited by']]

df_scopus = df_scopus.rename(columns={
    'Cited by': 'Citations'
})

df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/WACV57701.2024.00012,PECoP: Parameter Efficient Continual Pretraini...,"A., Dadashzadeh, Amirhossein; S., Duan, Shucha...",2024,The limited availability of labelled data in A...,15
1,10.1109/PRAI62207.2024.10827404,PE3DNet: A 'Pull-Up' Action Quality Assessment...,"R., Yang, Rui; X., Xiu, Xiaona; J., Wang, Jian...",2024,'Pull-up' is a regular physical fitness test i...,0
2,10.1109/ICAACE61206.2024.10548995,Action Quality Assessment with Multi-scale Tem...,"W., Wang, Wei; H., Wang, Hongyu; Y., Hao, Ying...",2024,Action quality assessment is a more challengin...,0
3,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,"T., Nagai, Takasuke; S., Takeda, Shoichiro; S....",2024,Action quality assessment (AQA) is a task for ...,2
4,10.1109/ICLT63507.2024.11038644,Online Deep Squat Evaluation: Leveraging Subje...,"S., Sardari, Sara; B., Nakisa, Bahareh; S., Sh...",2024,Evaluating deep squats accurately during autom...,0
5,10.1007/s10489-023-05166-3,Improving action quality assessment with acros...,"P., Lian, Puxiang; Z., Shao, Zhigang",2023,Action quality assessment is a significant res...,4
6,10.1109/TCSVT.2023.3281413,Hierarchical Graph Convolutional Networks for ...,"K., Zhou, Kanglei; Y., Ma, Yue; H.P., Shum, Hu...",2023,Action quality assessment (AQA) automatically ...,42
7,10.1016/j.heliyon.2023.e21361,The Establishment of a precise intelligent eva...,"N., Hao, Ning; S., Ruan, Sihan; Y., Song, Yihe...",2023,The introduction of action quality assessment ...,6
8,10.1145/3606038.3616150,Video-based Skill Assessment for Golf: Estimat...,"C.K., Ingwersen, Christian Keilstrup; A., Xarl...",2023,Automated skill assessment in sports using vid...,4
9,10.1145/3581783.3613774,A Figure Skating Jumping Dataset for Replay-Gu...,"Y., Liu, Yanchao; X., Cheng, Xina; T., Ikenaga...",2023,"In competitive sports, judges often scrutinize...",8


In [8]:
df_scopus = df_scopus.dropna(subset=['DOI'])
df_scopus = df_scopus.loc[df_scopus['Citations'] > 0]
df_scopus

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/WACV57701.2024.00012,PECoP: Parameter Efficient Continual Pretraini...,"A., Dadashzadeh, Amirhossein; S., Duan, Shucha...",2024,The limited availability of labelled data in A...,15
3,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,"T., Nagai, Takasuke; S., Takeda, Shoichiro; S....",2024,Action quality assessment (AQA) is a task for ...,2
5,10.1007/s10489-023-05166-3,Improving action quality assessment with acros...,"P., Lian, Puxiang; Z., Shao, Zhigang",2023,Action quality assessment is a significant res...,4
6,10.1109/TCSVT.2023.3281413,Hierarchical Graph Convolutional Networks for ...,"K., Zhou, Kanglei; Y., Ma, Yue; H.P., Shum, Hu...",2023,Action quality assessment (AQA) automatically ...,42
7,10.1016/j.heliyon.2023.e21361,The Establishment of a precise intelligent eva...,"N., Hao, Ning; S., Ruan, Sihan; Y., Song, Yihe...",2023,The introduction of action quality assessment ...,6
8,10.1145/3606038.3616150,Video-based Skill Assessment for Golf: Estimat...,"C.K., Ingwersen, Christian Keilstrup; A., Xarl...",2023,Automated skill assessment in sports using vid...,4
9,10.1145/3581783.3613774,A Figure Skating Jumping Dataset for Replay-Gu...,"Y., Liu, Yanchao; X., Cheng, Xina; T., Ikenaga...",2023,"In competitive sports, judges often scrutinize...",8
10,10.1145/3577190.3614117,MMASD: A Multimodal Dataset for Autism Interve...,"J., Li, Jicheng; V., Chheang, Vuthea; P., Kull...",2023,Autism spectrum disorder (ASD) is a developmen...,8
11,10.1109/TVCG.2023.3247092,A Video-Based Augmented Reality System for Hum...,"K., Zhou, Kanglei; R., Cai, Ruizhi; Y., Ma, Yu...",2023,As the most common idiopathic inflammatory myo...,16
12,10.1088/1742-6596/2632/1/012027,Prior Knowledge-guided Hierarchical Action Qua...,"H., Zhou, Haoyang; T., Hou, Teng; J., Li, Jitao",2023,"Recently, there has been a growing interest in...",1


### Total of 39 papers identified against 119 of the original paper
### *without DOMAIN filter, total of 110 papers

## Web of Science

In [9]:
df_wos = pd.read_excel(wos_file)
df_wos

,Publication Type,Authors,Book Authors,Group Authors,Book Group Authors,Researcher Ids,ORCIDs,Book Editors,Author - Arabic,Grant Principal Investigator,...,Copyright,Degree Name,Institution Address,Institution,Dissertation and Thesis Subjects,Author Keywords,Indexed Date,UT (Unique ID),Pubmed Id,Unnamed: 78
0,J,"Zhang, Boyu; Chen, Jiayuan; Xu, Yinfei; Zhang,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-24,WOS:001077141300004,NaN,NaN
1,J,"Nagai, Takasuke; Takeda, Shoichiro; Suzuki, Sa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-22,WOS:001269147100001,NaN,NaN
2,J,"Liu, Jiang; Wang, Huasheng; Stawarz, Katarzyna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-30,WOS:001363004900001,NaN,NaN
3,J,"Li, Ming-Zhe; Zhang, Hong-Bo; Dong, Li-Jia; Le...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-09,WOS:000874388900001,NaN,NaN
4,J,"Ren, Yuhao; Zhang, Bochao; Chen, Jing; Guo, Li...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-19,WOS:000866697300001,NaN,NaN
5,J,"Ke, Xiao; Xu, Huangbiao; Lin, Xiaofeng; Guo, W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-05-17,WOS:001218574700001,NaN,NaN
6,J,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-06,WOS:001123333700003,37963006.0,NaN
7,J,"Zhou, Kanglei; Cai, Ruizhi; Ma, Yue; Tan, Qing...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-02,WOS:000966597900001,37027743.0,NaN
8,J,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-11,WOS:001342519900003,39374293.0,NaN
9,J,"Zhou, Kanglei; Ma, Yue; Shum, Hubert P. H.; Li...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-02,WOS:001121618300065,NaN,NaN


In [10]:
df_wos = df_wos[['DOI', 'Article Title', 'Authors', 'Publication Year', 'Abstract', 'Times Cited, All Databases']]

df_wos = df_wos.rename(columns={
    'Article Title': 'Title',
    'Publication Year': 'Year',
    'Times Cited, All Databases': 'Citations'
})

df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1007/s00521-023-09068-w,Auto-encoding score distribution regression fo...,"Zhang, Boyu; Chen, Jiayuan; Xu, Yinfei; Zhang,...",2023,Assessing the quality of actions in videos is ...,19
1,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,"Nagai, Takasuke; Takeda, Shoichiro; Suzuki, Sa...",2024,Action quality assessment (AQA) is a task for ...,0
2,10.1016/j.eswa.2024.125642,Vision-based human action quality assessment: ...,"Liu, Jiang; Wang, Huasheng; Stawarz, Katarzyna...",2025,"Human Action Quality Assessment (AQA), which a...",2
3,10.1007/s40747-022-00892-6,Gaussian guided frame sequence encoder network...,"Li, Ming-Zhe; Zhang, Hong-Bo; Dong, Li-Jia; Le...",2023,Can a computer evaluate an athlete's performan...,6
4,10.3390/electronics11193051,An Efficient Motion Registration Method Based ...,"Ren, Yuhao; Zhang, Bochao; Chen, Jing; Guo, Li...",2022,Action quality assessment (AQA) is an importan...,0
5,10.1016/j.ins.2024.120347,Two-path target-aware contrastive regression f...,"Ke, Xiao; Xu, Huangbiao; Lin, Xiaofeng; Guo, W...",2024,Action quality assessment (AQA) is a challengi...,7
6,10.1109/TIP.2023.3331212,Fine-Grained Spatio-Temporal Parsing Network f...,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",2023,Action Quality Assessment (AQA) plays an impor...,13
7,10.1109/TVCG.2023.3247092,A Video-Based Augmented Reality System for Hum...,"Zhou, Kanglei; Cai, Ruizhi; Ma, Yue; Tan, Qing...",2023,As the most common idiopathic inflammatory myo...,18
8,10.1109/TIP.2024.3468870,Self-Supervised Sub-Action Parsing Network for...,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",2024,Semi-supervised Action Quality Assessment (AQA...,1
9,10.1109/TCSVT.2023.3281413,Hierarchical Graph Convolutional Networks for ...,"Zhou, Kanglei; Ma, Yue; Shum, Hubert P. H.; Li...",2023,Action quality assessment (AQA) automatically ...,38


In [11]:
df_wos = df_wos.dropna(subset=['DOI', 'Abstract'])
df_wos = df_wos.loc[df_wos['Year'] != 2025]
df_wos = df_wos.loc[df_wos['Citations'] > 0]
df_wos

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1007/s00521-023-09068-w,Auto-encoding score distribution regression fo...,"Zhang, Boyu; Chen, Jiayuan; Xu, Yinfei; Zhang,...",2023,Assessing the quality of actions in videos is ...,19
3,10.1007/s40747-022-00892-6,Gaussian guided frame sequence encoder network...,"Li, Ming-Zhe; Zhang, Hong-Bo; Dong, Li-Jia; Le...",2023,Can a computer evaluate an athlete's performan...,6
5,10.1016/j.ins.2024.120347,Two-path target-aware contrastive regression f...,"Ke, Xiao; Xu, Huangbiao; Lin, Xiaofeng; Guo, W...",2024,Action quality assessment (AQA) is a challengi...,7
6,10.1109/TIP.2023.3331212,Fine-Grained Spatio-Temporal Parsing Network f...,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",2023,Action Quality Assessment (AQA) plays an impor...,13
7,10.1109/TVCG.2023.3247092,A Video-Based Augmented Reality System for Hum...,"Zhou, Kanglei; Cai, Ruizhi; Ma, Yue; Tan, Qing...",2023,As the most common idiopathic inflammatory myo...,18
8,10.1109/TIP.2024.3468870,Self-Supervised Sub-Action Parsing Network for...,"Gedamu, Kumie; Ji, Yanli; Yang, Yang; Shao, Ji...",2024,Semi-supervised Action Quality Assessment (AQA...,1
9,10.1109/TCSVT.2023.3281413,Hierarchical Graph Convolutional Networks for ...,"Zhou, Kanglei; Ma, Yue; Shum, Hubert P. H.; Li...",2023,Action quality assessment (AQA) automatically ...,38
10,10.1007/s10489-024-05349-6,Assessing action quality with semantic-sequenc...,"Huang, Feng; Li, Jianjun",2024,Action Quality Assessment (AQA) is a critical ...,4
11,10.1007/s10489-023-04613-5,Multi-skeleton structures graph convolutional ...,"Lei, Qing; Li, Huiying; Zhang, Hongbo; Du, Jix...",2023,In most existing action quality assessment (AQ...,8
13,10.1109/TNSRE.2023.3317411,A Contrastive Learning Network for Performance...,"Yao, Long; Lei, Qing; Zhang, Hongbo; Du, Jixia...",2023,Human activity analysis in the legal monitorin...,11


### Total of 29 papers identified against 61 of the original paper
### *without 'Computer Vision' filter (domain), total of 54 papers

## Final Dataset

In [12]:
df_all = pd.concat([df_ieee, df_scopus, df_wos], ignore_index=True)
df_all

,DOI,Title,Authors,Year,Abstract,Citations
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...,1.0
1,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...,6.0
2,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...,86.0
3,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...,32.0
4,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...,2.0
...,...,...,...,...,...,...
87,10.1109/TCSVT.2020.3017727,Action Quality Assessment Using Siamese Networ...,"Jain, Hiteshi; Harit, Gaurav; Sharma, Avinash",2021,Automated vision-based score estimation models...,46.0
88,10.3390/electronics9040568,Learning Effective Skeletal Representations on...,"Lei, Qing; Zhang, Hong-Bo; Du, Ji-Xiang; Hsiao...",2020,"In this paper, we propose an integrated action...",16.0
89,10.1109/TMM.2023.3328180,Learning Semantics-Guided Representations for ...,"Du, Zexing; He, Di; Wang, Xue; Wang, Qing",2024,This paper explores semantic-aware representat...,9.0
90,10.3390/app142311130,Evaluation of Human Action Based on Feature-We...,"Yan, Mingdie; Liu, Xia; Li, Zhaoyang; Guo, Naiyu",2024,Action evaluation can automatically detect abn...,2.0


In [13]:
df_all = df_all.drop_duplicates(subset=['DOI'])
df_all = df_all.drop(columns=['Citations'])
df_all = df_all.reset_index(drop=True)
df_all

,DOI,Title,Authors,Year,Abstract
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...
1,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...
2,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...
3,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...
4,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...
...,...,...,...,...,...
62,10.1007/s11263-022-01695-5,Automatic Modelling for Interactive Action Ass...,"Gao, Jibin; Pan, Jia-Hui; Zhang, Shao-Jie; Zhe...",2023,"Action assessment, the task of visually assess..."
63,10.1109/TCSVT.2020.3017727,Action Quality Assessment Using Siamese Networ...,"Jain, Hiteshi; Harit, Gaurav; Sharma, Avinash",2021,Automated vision-based score estimation models...
64,10.3390/electronics9040568,Learning Effective Skeletal Representations on...,"Lei, Qing; Zhang, Hong-Bo; Du, Ji-Xiang; Hsiao...",2020,"In this paper, we propose an integrated action..."
65,10.3390/app142311130,Evaluation of Human Action Based on Feature-We...,"Yan, Mingdie; Liu, Xia; Li, Zhaoyang; Guo, Naiyu",2024,Action evaluation can automatically detect abn...


### Total of 67 papers identified against 91 of the original paper

## Step 2: Title + Abstract Evaluation with Gemini

In [14]:
# Load environment variables

load_dotenv()

MODEL= os.getenv("MODEL")
API_KEY = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=API_KEY)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [15]:
# Batch Evaluation

class BatchReviewItem(BaseModel):
    id: int
    decision: Annotated[Literal["yes", "no", "maybe"], Field(
        description="Decision about including the paper:\n"
                    "- 'yes': There is clear evidence that the article uses, shares, or discusses datasets, and it fully meets ALL inclusion criteria while violating NONE of the exclusion criteria.\n"
                    "- 'no': There is clear evidence that the article should be excluded because NOT ALL inclusion criteria are met or AT LEAST one exclusion criterion is violated.\n"
                    "- 'maybe': The information is insufficient or ambiguous.\n"
                    "If unsure, or it is needed to check other infos (e.g., full text) prefer 'maybe' over 'no'.")]
    reason: Annotated[str, Field(description="Short explanation for the decision")]
    data_source: Annotated[str, Field(
        description="Official and recognized name of the data source used in the paper if the dataset is explicitly mentioned.\n"
                    "Guidelines:\n"
                    "- The dataset must be a real, identifiable resource (dataset, database, or data collection); check online if necessary.\n"
                    "- If only a generic description is provided (e.g., 'agricultural data' or 'crop dataset' or 'generated data'), leave the field empty. However, if the description seems generic but is cited as a specific dataset, include it.\n"
                    "E.g., if the abstract sentence is: 'a crop profiles dataset', do not include it; if the sentence is: 'the crop profiles dataset', include it as a proper dataset name.\n"
                    "- If multiple data sources are mentioned, insert all of them in the same string separated by ';' (semicolon).\n"
                    "- If the decision is 'no', always leave it empty.")]


class BatchReview(BaseModel):
    items: list[BatchReviewItem]

fallback = BatchReviewItem(id=-1, decision="maybe", reason="No response", data_source="")


def is_open_data_relevant_batch(df_chunk: pd.DataFrame, client, model=None, max_retries=3):
    if model is None:
        model = MODEL if MODEL else "gemini-2.5-flash-lite"

    # Prepare record strings for prompt
    records = []
    for idx, row in df_chunk.iterrows():
        title = "" if pd.isna(row.get("Title")) else str(row.get("Title"))
        abstract = "" if pd.isna(row.get("Abstract")) else str(row.get("Abstract"))
        records.append(f"- id: {int(idx)}\n  Title: {title}\n  Abstract: {abstract}")

    # Build prompt with inclusion/exclusion criteria and instructions
    prompt = (
            "Evaluate each record based on the following Inclusion and Exclusion Criteria and return ONLY JSON that matches the provided schema.\n\n"
            "Inclusion Criteria:\n"
            "(a) The article mentions the use or production of datasets, databases, or other data sources.\n"
            "(b) The article is related to computer science.\n"
            "Exclusion Criteria:\n"
            "(a) The article states that the data are not available, must be requested from the authors or are not free (cost is required)."
            "(b) The article presents results of surveys or review.\n"
            "(c) The article is not related to using vision-based methods.\n"
            "(d) The article is not related to Human Action Quality Assessment (AQA).\n"
            "ALL the requirements should be met.\n\n"
            "Output Instructions:\n"
            "- For each provided record, return an object with: id, decision ('yes'|'no'|'maybe'), reason, data_source.\n"
            "- If title or abstract are missing or empty use decision='no' and reason='Title or abstract is missing', data_source=''.\n\n"
            "Records to evaluate:\n"
            + "\n".join(records)
    )

    global _sent_calls_count
    global _completed_evaluations_count
    backoff = 10

    # Error management with exponential backoff
    for _ in range(max_retries):
        _rate_limit_block_until_allowed()
        try:
            _sent_calls_count += 1
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": BatchReview,
                    "temperature": 0.0
                },
            )
            # Update total evaluations count
            try:
                _completed_evaluations_count += len(df_chunk)
            except Exception as e:
                print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
                _completed_evaluations_count = len(df_chunk)
            print(f"[GEMINI] Completed {_completed_evaluations_count} total evaluations.")
            return resp.parsed.items
        except Exception as e:
            print(f"[attempt {_sent_calls_count}] Gemini error: {e}")
            time.sleep(backoff)
            backoff *= 2
    return []


def _rate_limit_block_until_allowed():
    # Block until the number of calls in the window is below max
    while True:
        now = time.monotonic()
        while _rate_calls_log and (now - _rate_calls_log[0]) >= _RATE_WINDOW_SEC:
            _rate_calls_log.popleft()
        if len(_rate_calls_log) < _RATE_MAX_CALLS:
            _rate_calls_log.append(now)
            return


# Rate limiter: max 2 requests every 60 seconds
_RATE_WINDOW_SEC = 60.0
_RATE_MAX_CALLS = 2
_rate_calls_log = deque()

batch_size = 20  # number of papers per batch
df_evaluated = df_all.copy()

_sent_calls_count = 0
_completed_evaluations_count = 0

# Process dataframe in batches
for start in range(0, len(df_evaluated), batch_size):
    chunk = df_evaluated.iloc[start:start + batch_size]
    items = is_open_data_relevant_batch(chunk, client, model=MODEL)
    mapping = {int(it.id): it for it in items} if items else {}

    decisions = []
    reasons = []
    sources = []
    for idx in chunk.index:
        it = mapping.get(int(idx))
        if it is None:
            decisions.append(fallback.decision)
            reasons.append(fallback.reason)
            sources.append(fallback.data_source)
        else:
            decisions.append(it.decision)
            reasons.append(it.reason)
            sources.append(it.data_source)

    # Update dataframe with results
    df_evaluated.loc[chunk.index, 'Include'] = decisions
    df_evaluated.loc[chunk.index, 'Reason'] = reasons
    df_evaluated.loc[chunk.index, 'Data Source'] = sources

df_evaluated

[GEMINI] Completed 20 total evaluations.
[GEMINI] Completed 40 total evaluations.
[attempt 3] Gemini error: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
[attempt 4] Gemini error: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
[GEMINI] Completed 60 total evaluations.
[attempt 6] Gemini error: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
[GEMINI] Completed 67 total evaluations.


,DOI,Title,Authors,Year,Abstract,Include,Reason,Data Source
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...,yes,The paper introduces a new multimodal dataset ...,MMW-AQA
1,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...,yes,The paper proposes a new vision-based method f...,
2,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...,yes,"The paper introduces a new, publicly available...",FineDiving
3,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...,yes,The paper constructs and releases a new long-f...,LOGO
4,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...,yes,The paper applies Action Quality Assessment (A...,
...,...,...,...,...,...,...,...,...
62,10.1007/s11263-022-01695-5,Automatic Modelling for Interactive Action Ass...,"Gao, Jibin; Pan, Jia-Hui; Zhang, Shao-Jie; Zhe...",2023,"Action assessment, the task of visually assess...",maybe,The paper proposes a new framework for Action ...,JIGSAWS; TASD-2; PaSk
63,10.1109/TCSVT.2020.3017727,Action Quality Assessment Using Siamese Networ...,"Jain, Hiteshi; Harit, Gaurav; Sharma, Avinash",2021,Automated vision-based score estimation models...,no,The abstract describes a new model for action ...,
64,10.3390/electronics9040568,Learning Effective Skeletal Representations on...,"Lei, Qing; Zhang, Hong-Bo; Du, Ji-Xiang; Hsiao...",2020,"In this paper, we propose an integrated action...",yes,The paper proposes a method for Human Action Q...,MIT Olympic Scoring dataset; University of Nev...
65,10.3390/app142311130,Evaluation of Human Action Based on Feature-We...,"Yan, Mingdie; Liu, Xia; Li, Zhaoyang; Guo, Naiyu",2024,Action evaluation can automatically detect abn...,maybe,The paper discusses an action evaluation syste...,


In [ ]:
# Review Process

class HarmonizedSources(BaseModel):
    sources: list[str] = Field(
        description="Cleaned and harmonized list of data sources, same order as input"
    )

def harmonize_all_data_sources(df: pd.DataFrame, client, model=None, max_retries=3):
    if model is None:
        model = MODEL if MODEL else "gemini-2.5-flash-lite"

    # Prepare inputs
    records = []
    for idx, row in df.iterrows():
        sources = "" if pd.isna(row.get("Data Source")) else str(row.get("Data Source"))
        records.append(f"- id: {int(idx)}\n  Data Source: {sources}")

    # Prompt unico
    prompt = (
        "Check and harmonize all dataset names for consistency and correctness.\n"
        "Inclusion/Exclusion Guidelines:\n"
        "- The dataset must be related to Human Action Quality Assessment (AQA).\n"
        "- The dataset must be publicly available (open access).\n"
        "- If a dataset does not meet these criteria, leave the field empty.\n"
        "Format Guidelines:\n"
        "- Remove descriptors like 'dataset', 'database', 'data'.\n"
        "- If multiple datasets are listed, leave the separation with ';' (semicolon).\n"
        "- Always capitalize the first letter of each dataset.\n"
        "- Prefer the recognized abbreviation if it exists, otherwise use the official full name. Do not invent abbreviations.\n"
        "- Ensure consistent naming across all rows, without forcing or inventing associations. Associate only if you are sure.\n"
        "- If a field is empty, keep it empty.\n\n"
        "Output Instructions:\n"
        "- Return only JSON matching the schema.\n"
        "- Return exactly one harmonized string per input row, in the same order as provided.\n\n"
        "Records to harmonize:\n"
        + "\n".join(records)
    )

    backoff = 10
    for _ in range(max_retries):
        _rate_limit_block_until_allowed()
        try:
            resp = client.models.generate_content(
                model=model,
                contents=prompt,
                config={
                    "response_mime_type": "application/json",
                    "response_schema": HarmonizedSources,
                    "temperature": 0.0
                },
            )
            return resp.parsed.sources
        except Exception as e:
            print(f"[GEMINI ERROR] {e}")
            time.sleep(backoff)
            backoff *= 2
    return ["" for _ in df.index]

df_evaluated["Data Source Harmonized"] = harmonize_all_data_sources(df_evaluated, client, model=MODEL)
df_evaluated

[harmonization error] 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
[harmonization error] 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
[harmonization error] 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}


In [17]:
df_evaluated

,DOI,Title,Authors,Year,Abstract,Include,Reason,Data Source
0,10.1109/ACCESS.2024.3423462,MMW-AQA: Multimodal In-the-Wild Dataset for Ac...,T. Nagai; S. Takeda; S. Suzuki; H. Seshimo,2024,Action quality assessment (AQA) is a task for ...,yes,The paper introduces a new dataset called MMW-...,MMW-AQA
1,10.1109/ICIP42928.2021.9506257,Action Quality Assessment With Ignoring Scene ...,T. Nagai; S. Takeda; M. Matsumura; S. Shimizu;...,2021,We propose an action quality assessment (AQA) ...,yes,The paper proposes a new method for Action Qua...,
2,10.1109/CVPR52688.2022.00296,FineDiving: A Fine-grained Dataset for Procedu...,J. Xu; Y. Rao; X. Yu; G. Chen; J. Zhou; J. Lu,2022,Most existing action quality assessment method...,yes,"The paper introduces a new, publicly available...",FineDiving
3,10.1109/CVPR52729.2023.00238,LOGO: A Long-Form Video Dataset for Group Acti...,S. Zhang; W. Dai; S. Wang; X. Shen; J. Lu; J. ...,2023,Action quality assessment (AQA) has become an ...,yes,The paper introduces a new dataset named LOGO ...,LOGO
4,10.1109/ICMLC58545.2023.10327994,Action Quality Assessment for ASD Behaviour Ev...,D. Zhang; D. Zhou; H. Liu,2023,Given the current increasing prevalence of aut...,yes,"The paper applies Action Quality Assessment, a...",
...,...,...,...,...,...,...,...,...
62,10.1007/s11263-022-01695-5,Automatic Modelling for Interactive Action Ass...,"Gao, Jibin; Pan, Jia-Hui; Zhang, Shao-Jie; Zhe...",2023,"Action assessment, the task of visually assess...",yes,The paper proposes a new framework for vision-...,JIGSAWS;TASD-2;PaSk;AQA-7
63,10.1109/TCSVT.2020.3017727,Action Quality Assessment Using Siamese Networ...,"Jain, Hiteshi; Harit, Gaurav; Sharma, Avinash",2021,Automated vision-based score estimation models...,yes,The paper proposes a new vision-based system f...,
64,10.3390/electronics9040568,Learning Effective Skeletal Representations on...,"Lei, Qing; Zhang, Hong-Bo; Du, Ji-Xiang; Hsiao...",2020,"In this paper, we propose an integrated action...",yes,The paper proposes a vision-based framework fo...,MIT-Skate;UNIV-Dive
65,10.3390/app142311130,Evaluation of Human Action Based on Feature-We...,"Yan, Mingdie; Liu, Xia; Li, Zhaoyang; Guo, Naiyu",2024,Action evaluation can automatically detect abn...,yes,The paper proposes a vision-based method for e...,


In [16]:
# File Export

df_evaluated["Data Source"] = df_evaluated["Data Source Harmonized"]
df_evaluated.drop(columns=["Data Source Harmonized"], inplace=True)

df_evaluated.to_csv("data/validation/df_evaluated_validation.csv", index=False)